# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [ ]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [17]:
# 1) Seu código aqui

# URL da API para carregar os dados de PVLs
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"

# Fazer a requisição para a API
response = requests.get(url)

# Verificar se a resposta foi bem sucedida
if response.status_code == 200:
    # Transformar os dados JSON em um DataFrame do pandas
    data = response.json()
    df = pd.DataFrame(data['items'])
    
    # Exibir as colunas disponíveis no DataFrame para verificação
    print("Colunas disponíveis no DataFrame:", df.columns)
    
    # Verificar as primeiras linhas do DataFrame
    print("Amostra dos dados:", df.head())

# Filtrar os PVLs realizados por municípios no estado do Rio de Janeiro
df_rj_municipios = df[(df['tipo_interessado'] == 'Município') & (df['uf'] == 'RJ')]

# Exibir os primeiros registros filtrados
print(df_rj_municipios.head())

Colunas disponíveis no DataFrame: Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status'],
      dtype='object')
Amostra dos dados:    id_pleito tipo_interessado interessado  cod_ibge  uf num_pvl  \
0      11431           Estado       Goiás        52  GO    None   
1      13318        Município    Mesquita   3141702  MG    None   
2      13337        Município    Pitangui   3151404  MG    None   
3      10654           Estado   Tocantins        17  TO    None   
4      10668           Estado    Amazonas        13  AM    None   

                                              status          num_processo  \
0  Encaminhado à PGFN com manifestação técnica fa...  17944.001445/2012-00   
1                                           Deferido  17944.002023/2013-24   
2   

In [9]:
# 2) Seu código aqui
# Contar a frequência dos diferentes status
status_frequencia = df_rj_municipios['status'].value_counts()

# Exibir os três status mais frequentes
top_3_status = status_frequencia.head(3)
print("Os três status mais frequentes e suas frequências são:")
print(top_3_status)


Os três status mais frequentes e suas frequências são:
status
Deferido             26
Arquivado            10
Deferido (PVL-IF)     5
Name: count, dtype: int64


In [15]:
# 3) Seu código aqui
# Converter 'data_status' para string e extrair o ano
df_rj_municipios.loc[:, 'ano_status'] = df_rj_municipios['data_status'].str.slice(start=-4)

# Exibir as primeiras linhas para verificar a nova coluna
print(df_rj_municipios[['data_status', 'ano_status']].head())

    data_status ano_status
171  15/02/2007       2007
191  02/06/2020       2020
247  13/03/2007       2007
344  05/10/2023       2023
356  27/06/2024       2024


In [26]:
# 4) Certifique-se de que 'ano_status' foi criada corretamente usando .loc
df_rj_municipios.loc[:, 'ano_status'] = df_rj_municipios['data_status'].str.slice(start=-4)

# Agora, conte a frequência de cada ano
frequencia_ano_status = df_rj_municipios['ano_status'].value_counts()

# Exibir as frequências
print("Frequência de cada ano no campo 'ano_status':")
print(frequencia_ano_status)




Frequência de cada ano no campo 'ano_status':
ano_status
2007    9
2008    8
2014    6
2023    5
2022    5
2010    4
2019    4
2015    3
2011    2
2017    2
2004    1
2003    1
2013    1
2012    1
2020    1
2006    1
2024    1
2009    1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [28]:
import requests
import pandas as pd

# URL da API para carregar os dados de PVLs
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"

# Fazer a requisição para a API
response = requests.get(url)

# Verificar se a resposta foi bem sucedida
if response.status_code == 200:
    # Transformar os dados JSON em um DataFrame do pandas
    data = response.json()
    df = pd.DataFrame(data['items'])
    
    # Exibir as colunas disponíveis no DataFrame para verificação
    print("Colunas disponíveis no DataFrame:", df.columns)
    
    # Verificar as primeiras linhas do DataFrame
    print("Amostra dos dados:", df.head())
    
    # Verificar se as colunas 'ente' e 'uf' estão presentes antes de filtrar
    if 'ente' in df.columns and 'uf' in df.columns:
        # Filtrar os dados para selecionar apenas os PVLs de municípios no estado do RJ
        df_rj = df[(df['ente'] == 'Município') & (df['uf'] == 'RJ')]
        print("Filtragem realizada com sucesso")
    else:
        print("As colunas 'ente' ou 'uf' não estão presentes nos dados.")
else:
    print(f"Erro na requisição: {response.status_code}")


Colunas disponíveis no DataFrame: Index(['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf',
       'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao',
       'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor',
       'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status'],
      dtype='object')
Amostra dos dados:    id_pleito tipo_interessado interessado  cod_ibge  uf num_pvl  \
0      11431           Estado       Goiás        52  GO    None   
1      13318        Município    Mesquita   3141702  MG    None   
2      13337        Município    Pitangui   3151404  MG    None   
3      10654           Estado   Tocantins        17  TO    None   
4      10668           Estado    Amazonas        13  AM    None   

                                              status          num_processo  \
0  Encaminhado à PGFN com manifestação técnica fa...  17944.001445/2012-00   
1                                           Deferido  17944.002023/2013-24   
2   

In [30]:
# 1) Seu código aqui

def consultar_api(uf, tipo_interessado):
    # URL da API
    url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    
    # Fazer a requisição para a API
    response = requests.get(url)
    
    # Verificar se a resposta foi bem sucedida
    if response.status_code == 200:
        # Transformar os dados JSON em um DataFrame do pandas
        data = response.json()
        df = pd.DataFrame(data['items'])
        
        # Filtrar os dados pelo tipo de interessado e UF
        df_filtered = df[(df['tipo_interessado'] == tipo_interessado) & (df['uf'] == uf)]
        return df_filtered
    else:
        print(f"Erro na requisição: {response.status_code}")
        return None


In [32]:
# 2) Seu código aqui
# Usar a função para obter dados para Minas Gerais com o tipo 'Estado'
df_mg_estados = consultar_api('MG', 'Estado')

# Contar quantas solicitações estão arquivadas por decurso de prazo
num_archived_requests = df_mg_estados[df_mg_estados['status'] == 'Arquivado por decurso de prazo'].shape[0]

print(f"Número de solicitações para o Estado de Minas Gerais com status 'Arquivado por decurso de prazo': {num_archived_requests}")


Número de solicitações para o Estado de Minas Gerais com status 'Arquivado por decurso de prazo': 0


In [34]:
# 3) Seu código aqui
# Usar a função para obter dados para a Bahia com o tipo 'Município'
df_bahia_municipios = consultar_api('BA', 'Município')

# Contar as solicitações deferidas por município
deferidos_counts = df_bahia_municipios[df_bahia_municipios['status'] == 'Deferido']['interessado'].value_counts()

# Obter o município com mais solicitações deferidas
municipio_mais_deferido = deferidos_counts.idxmax()
quantidade_deferidos = deferidos_counts.max()

print(f"O município da Bahia com mais solicitações deferidas é {municipio_mais_deferido} com {quantidade_deferidos} solicitações.")


O município da Bahia com mais solicitações deferidas é Luís Eduardo Magalhães com 3 solicitações.


In [38]:
# 4) Filtrar os dados da Bahia com interessado 'Estado'
df_bahia_estados = df_bahia_municipios[df_bahia_municipios['tipo_interessado'] == 'Estado']

# Salvar em um arquivo .csv
df_bahia_estados.to_csv('solicitacoes_bahia_estado.csv', index=False)

print("Os dados foram salvos no arquivo 'solicitacoes_bahia_estado.csv'.")


Os dados foram salvos no arquivo 'solicitacoes_bahia_estado.csv'.
